# Semantic Segmentation with pretrained models
----------------------

This notebook implements an alternative training approach including the usage of pretrained models.
These models (e.g. resnet34) replace the encoder part of the standard Unet architecture.

--Warning-- There are still some dimensionality issues (RGB vs grayscale), which are not ironed out

In [ ]:
# Import packages
import segmentation_models as sm
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow
import boto3
import mlflow

sm.set_framework('tf.keras')
sm.framework()

In [ ]:
# Load train data
image_names = glob.glob("../data/data_train/train/images/train/*.tif")
image_names.sort()
images = [cv2.imread(image, 1) for image in image_names] #SM backbones use 3 channel images, so let us read images in color.
image_dataset = np.array(images)

mask_names = glob.glob("../data/data_train/train/masks/train/*.tif")
mask_names.sort()
masks = [cv2.imread(mask, 0) for mask in mask_names]
mask_dataset = np.array(masks)

# Load val data
image_names = glob.glob("../data/data_train/val/images/val/*.tif")
image_names.sort()
images = [cv2.imread(image, 1) for image in image_names] #SM backbones use 3 channel images, so let us read images in color.
val_image_dataset = np.array(images)

mask_names = glob.glob("../data/data_train/val/masks/val/*.tif")
mask_names.sort()
masks = [cv2.imread(mask, 0) for mask in mask_names]
val_mask_dataset = np.array(masks)

In [ ]:
#Encode labels to 0, 1, 2, 3, ... but multi dim array so need to flatten, encode and reshape
labelencoder = LabelEncoder()
n, h, w = mask_dataset.shape  
mask_dataset_reshaped = mask_dataset.reshape(-1,1)
mask_dataset_reshaped_encoded = labelencoder.fit_transform(mask_dataset_reshaped)
mask_dataset_encoded = mask_dataset_reshaped_encoded.reshape(n, h, w)
n_classes=4

labelencoder = LabelEncoder()
n, h, w = val_mask_dataset.shape  
mask_dataset_reshaped = val_mask_dataset.reshape(-1,1)
mask_dataset_reshaped_encoded = labelencoder.fit_transform(mask_dataset_reshaped)
val_mask_dataset_encoded = mask_dataset_reshaped_encoded.reshape(n, h, w)
n_classes=4

In [ ]:
mask_dataset_encoded = np.expand_dims(mask_dataset_encoded, axis = 3)
val_mask_dataset_encoded = np.expand_dims(val_mask_dataset_encoded, axis = 3)

In [ ]:
#Normalize images
image_dataset = image_dataset /255.  #Can also normalize or scale using MinMax scaler
val_image_dataset = val_image_dataset /255.  #Can also normalize or scale using MinMax scaler

In [ ]:
train_masks_cat = to_categorical(mask_dataset_encoded, num_classes=n_classes)
train_masks_cat = train_masks_cat.reshape((mask_dataset_encoded.shape[0], mask_dataset_encoded.shape[1], mask_dataset_encoded.shape[2], n_classes))

val_masks_cat = to_categorical(val_mask_dataset_encoded, num_classes=n_classes)
val_masks_cat = val_masks_cat.reshape((val_mask_dataset_encoded.shape[0], val_mask_dataset_encoded.shape[1], val_mask_dataset_encoded.shape[2], n_classes))

In [ ]:
#Reused parameters in all models
n_classes=4
activation='softmax'

LR = 0.0001
optim = tensorflow.keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
###Model #for resnet use resnet34
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# preprocess input
train_processed = preprocess_input(image_dataset)
val_processed = preprocess_input(val_image_dataset)

In [ ]:
from keras.layers import Input, Conv2D
from keras.models import Model

# Do some stuff because our input images have only one channel
N = train_processed.shape[-1] # 1 in our case because grayscale
base_model= sm.Unet(BACKBONE, encoder_weights='imagenet')

inp = Input(shape=(512,512,N))
l1 = Conv2D(3, (1,1))(inp) # map N channels to 3 channels
out = base_model(l1)

model = Model(inp, out, name=base_model.name)

In [ ]:
# # define model
# model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics=metrics)

In [ ]:
# ML_FLOW
epochs = 40
batch_size = 4
model_type = 'resnet34'
loss_name = 'total_loss'

model_name = f'{model_type}_{loss_name}_epochs{epochs}_batchsize{batch_size}_learningrate{LR}'
TRACKING_URI = open('../.mlflow_uri').read().strip()
mlflow.set_tracking_uri(TRACKING_URI)

# Set an experiment name, which must be unique and case-sensitive.
experiment = mlflow.set_experiment('magic rocks!')

# Set run name and start run
mlflow.start_run(run_name=f'{model_name}')
mlflow.tensorflow.autolog()

# Get Experiment Details
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Name: {}".format(experiment.name))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

In [ ]:
history=model.fit(train_processed, 
          train_masks_cat,
          batch_size=batch_size, 
          epochs=epochs,
          verbose=1,
          validation_data=(val_processed, val_masks_cat))

In [ ]:
# End mlflow run
mlflow.end_run()

In [ ]:
model.save(f'../models/{model_name}.hdf5')

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig(f'../models/{model_name}.jpg', dpi=150)